# Viewing categorical AIS tracks 


This notebook prototypes the use categorical datashading of AIS data.

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import colorcet
import datashader as ds
import holoviews as hv
from holoviews.operation.datashader import rasterize, datashade, dynspread
hv.extension('bokeh')

Defining categories by collapsing the 100 [AIS Vessel Types](https://api.vtexplorer.com/docs/ref-aistypes.html) down to 22 broader vessel types:

In [ ]:
groups = {0:'Unknown', 1:'WIG', 2:'Fishing', 3:'Towing', 4:'Dredging', 5:'Diving', 6:'Military', 
          7:'Sailing', 8:'Pleasure', 9:'High Speed', 10:'Pilot Vessel', 11:'Search and Rescue vessel',
          12:'Tug', 13:'Port Tender', 14:'Anti-pollution equipment', 15:'Law Enforcement', 16:'Spare',
          17:'Medical Transport', 18:'Noncombatant', 19:'Passenger', 20:'Cargo', 21:'Tanker', 22:'Other'}
                 
def group_categories(vtype):
    if vtype == 0 or vtype==38 or vtype==39: # 'Unavailable' or 'Reserved'
        return 0
    if 19 < vtype < 30:
        return 1
    if vtype == 30:
        return 2
    if vtype == 31 or vtype == 32:
        return 3
    if vtype==33:
        return 4
    if vtype==34:
        return 5
    if vtype==35:
        return 6
    if vtype==36:
        return 7
    if vtype==37:
        return 8
    if 39 < vtype < 50:
        return 9
    mapping = {50:10, 51:11,52:12, 53:13, 54:14, 55:15, 56:16,  57:16, 58:17, 59:18}
    match = mapping.get(vtype, None)
    if match is not None:
        return match
    if 59 < vtype < 70:
        return 19
    if 69 < vtype < 80:
        return 20
    if 79 < vtype < 90:
        return 21
    if 89 < vtype < 100:
        return 22

## Loading AIS pings and Vessel information

In [ ]:
broadcast = pd.read_csv('../Broadcast.csv', parse_dates=[1]) # Zone 10
vessel_info =  pd.read_csv('../Vessel.csv')
vessel_info['vessel_type']= vessel_info['vessel_type'].fillna(0).astype(int) # Mapping NaN to 'Not Available' i.e 0
broadcast.head()

## Assigning broader categories

In [ ]:
vessel_mapping = {k:v for k,v in zip(vessel_info['mmsi_id'], 
                                     vessel_info['vessel_type'].apply(group_categories))} # Dict for mmsi:vessel type
# Join between broadcast AIS pings and vessel info
categories = broadcast['mmsi_id'].apply(lambda x: vessel_mapping.get(x, 0))
broadcast['category'] = categories

### Defining color key and legend

In [ ]:
def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb

color_key = {ind:tuple(int(el*255.) for el in val) for ind,val in 
             enumerate(colorcet.glasbey_bw_minc_20_hue_330_100[:23])} # 23 Glasby colors
color_names = {groups[k]:rgb_to_hex(v) for k,v in color_key.items()}
color_points = hv.NdOverlay({k: hv.Points([0,0], label=str(k)).opts(color=v, size=0) for k, v in color_names.items()})

In [ ]:
eastings, northings = zip(*[hv.util.transform.lon_lat_to_easting_northing(lon, lat) for lon, lat 
                          in zip(broadcast['lon'], broadcast['lat'])])

# Datashaded, categorical AIS plot (Zone 10)

In [ ]:
tiles = hv.element.tiles.ESRI().redim(x='easting', y='northing')
tiles.opts( width=900, height=620) * color_points * dynspread(datashade(hv.Points(pd.DataFrame({'northing':northings, 
                                  'easting':eastings, 'category':categories}),
                    ['easting', 'northing'], ['category']), color_key=color_key, 
                                                            aggregator=ds.count_cat('category')))

## Full AIS vessel type mapping

The following is not used in this notebook but allows Pythonic access to the full set of AIS vessel types:

In [ ]:
mapping = {
    0:'Not available',
    20:'Wing in ground (WIG), all ships of this type',
    21:'Wing in ground (WIG), Hazardous category A',
    22:'Wing in ground (WIG), Hazardous category B',
    23:'Wing in ground (WIG), Hazardous category C',
    24:'Wing in ground (WIG), Hazardous category D',
    25:'Wing in ground (WIG), Reserved for future use',
    26:'Wing in ground (WIG), Reserved for future use',
    27:'Wing in ground (WIG), Reserved for future use',
    28:'Wing in ground (WIG), Reserved for future use',
    29:'Wing in ground (WIG), Reserved for future use',
    30:'Fishing',
    31:'Towing',
    32:'Towing: length exceeds 200m or breadth exceeds 25m',
    33:'Dredging or underwater ops',
    34:'Diving ops',
    35:'Military ops',
    36:'Sailing',
    37:'Pleasure Craft',
    38:'Reserved',
    39:'Reserved',
    40:'High speed craft (HSC), all ships of this type',
    41:'High speed craft (HSC), Hazardous category A',
    42:'High speed craft (HSC), Hazardous category B',
    43:'High speed craft (HSC), Hazardous category C',
    44:'High speed craft (HSC), Hazardous category D',
    45:'High speed craft (HSC), Reserved for future use',
    46:'High speed craft (HSC), Reserved for future use',
    47:'High speed craft (HSC), Reserved for future use',
    48:'High speed craft (HSC), Reserved for future use',
    49:'High speed craft (HSC), No additional information',
    50:'Pilot Vessel',
    51:'Search and Rescue vessel',
    52:'Tug',
    53:'Port Tender',
    54:'Anti-pollution equipment',
    55:'Law Enforcement',
    56:'Spare - Local Vessel',
    57:'Spare - Local Vessel',
    58:'Medical Transport',
    59:'Noncombatant ship according to RR Resolution No. 18',
    60:'Passenger, all ships of this type',
    61:'Passenger, Hazardous category A',
    62:'Passenger, Hazardous category B',
    63:'Passenger, Hazardous category C',
    64:'Passenger, Hazardous category D',
    65:'Passenger, Reserved for future use',
    66:'Passenger, Reserved for future use',
    67:'Passenger, Reserved for future use',
    68:'Passenger, Reserved for future use',
    69:'Passenger, No additional information',
    70:'Cargo, all ships of this type',
    71:'Cargo, Hazardous category A',
    72:'Cargo, Hazardous category B',
    73:'Cargo, Hazardous category C',
    74:'Cargo, Hazardous category D',
    75:'Cargo, Reserved for future use',
    76:'Cargo, Reserved for future use',
    77:'Cargo, Reserved for future use',
    78:'Cargo, Reserved for future use',
    79:'Cargo, No additional information',
    80:'Tanker, all ships of this type',
    81:'Tanker, Hazardous category A',
    82:'Tanker, Hazardous category B',
    83:'Tanker, Hazardous category C',
    84:'Tanker, Hazardous category D',
    85:'Tanker, Reserved for future use',
    86:'Tanker, Reserved for future use',
    87:'Tanker, Reserved for future use',
    88:'Tanker, Reserved for future use',
    89:'Tanker, No additional information',
    90:'Other Type, all ships of this type',
    91:'Other Type, Hazardous category A',
    92:'Other Type, Hazardous category B',
    93:'Other Type, Hazardous category C',
    94:'Other Type, Hazardous category D',
    95:'Other Type, Reserved for future use',
    96:'Other Type, Reserved for future use',
    97:'Other Type, Reserved for future use',
    98:'Other Type, Reserved for future use',
    99:'Other Type, no additional information'}

def vessel_type_from_int(val):
    val = int(val)
    name = mapping.get(val, None)
    if name is not None:
        return name
    if val < 20:
        return 'Reserved for future use'
    else:
        return 'Unknown vessel type'